## 1. ML flow settings

- pip install mlflow
- mlflow ui

## 2. Model Load

In [17]:
# 라이브러리 import
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import mlflow
import mlflow.sklearn

In [18]:
iris = load_iris() # 꽃 받침과 꽃 잎 사이즈를 가지고 꽃의 종류를 결정

X = iris.data
y = iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습 데이터와 테스트 데이터로 분리 => train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=123)

In [19]:
from sklearn.linear_model import LogisticRegression

LogisticRegression?

Init signature:
LogisticRegression(
    penalty='l2',
    *,
    dual=False,
    tol=0.0001,
    C=1.0,
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    random_state=None,
    solver='lbfgs',
    max_iter=100,
    multi_class='deprecated',
    verbose=0,
    warm_start=False,
    n_jobs=None,
    l1_ratio=None,
)
Docstring:     
Logistic Regression (aka logit, MaxEnt) classifier.

In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
scheme if the 'multi_class' option is set to 'ovr', and uses the
cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
(Currently the 'multinomial' option is supported only by the 'lbfgs',
'sag', 'saga' and 'newton-cg' solvers.)

This class implements regularized logistic regression using the
'liblinear' library, 'newton-cg', 'sag', 'saga' and 'lbfgs' solvers. **Note
that regularization is applied by default**. It can handle both dense
and sparse input. Use C-ordered arrays or CSR matrices co

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=0)
model.fit(X_train, y_train) # train=모의고사 # 학습을 시킬 때는 학습 데이터만 제공

y_pred = model.predict(X_test) # 수능 문제를 제공
accuracy = accuracy_score(y_test, y_pred)

print(f"정확도 : {accuracy * 100}")

2024/08/14 15:42:49 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6475038260914396a152a4b00f8af678', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
/Users/a-08/mlflow/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/08/14 15:42:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run blushing-bat-446 at: http://127.0.0.1:5000/#/experiments/0/runs/6475038260914396a152a4b00f8af678.
2024/08/14 15:42:51 INFO mlflow.tracki

정확도 : 93.33333333333333


In [21]:
from sklearn.metrics import accuracy_score
accuracy_score?

Signature: accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)
Docstring:
Accuracy classification score.

In multilabel classification, this function computes subset accuracy:
the set of labels predicted for a sample must *exactly* match the
corresponding set of labels in y_true.

Read more in the :ref:`User Guide <accuracy_score>`.

Parameters
----------
y_true : 1d array-like, or label indicator array / sparse matrix
    Ground truth (correct) labels.

y_pred : 1d array-like, or label indicator array / sparse matrix
    Predicted labels, as returned by a classifier.

normalize : bool, default=True
    If ``False``, return the number of correctly classified samples.
    Otherwise, return the fraction of correctly classified samples.

sample_weight : array-like of shape (n_samples,), default=None
    Sample weights.

Returns
-------
score : float or int
    If ``normalize == True``, return the fraction of correctly
    classified samples (float), else returns the numb

### 모델 학습과 모델 성능

- 심플하게 모든 것을 ML flow에게 맡긴다. => mlflow.autolog()
- autolog에서 추적하지 못하는 다른 파라미터, 메트릭, 메타데이터 등의 값을 수동으로 기록

In [22]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print("Tracking URI", mlflow.get_tracking_uri())

Tracking URI http://127.0.0.1:5000


In [23]:
exp = mlflow.set_experiment(experiment_name='iris_classification_experiment')

print(f"Name: {exp.name}") # option + shift + 화살표 = copy
print(f"ID: {exp.experiment_id}")
print(f"Location: {exp.artifact_location}")
print(f"Tags: {exp.tags}")
print(f"Lifecycle: {exp.lifecycle_stage}")
print(f"Create Timestamp: {exp.creation_time}")


Name: iris_classification_experiment
ID: 471758380148410695
Location: mlflow-artifacts:/471758380148410695
Tags: {}
Lifecycle: active
Create Timestamp: 1723613298348


In [24]:
import time

time.time()

1723617783.492173

In [25]:
import mlflow.sklearn

mlflow.autolog()

mlflow.start_run() # 실험 시작
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train) # 학습 시킬 때는 학습 데이터만 제공

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"accuracy: {accuracy * 100}")

mlflow.end_run() # 실험 종료

2024/08/14 15:43:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/08/14 15:43:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run skittish-zebra-509 at: http://127.0.0.1:5000/#/experiments/471758380148410695/runs/710be8818e364beba1e429da28f87f03.
2024/08/14 15:43:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/471758380148410695.


accuracy: 96.66666666666667


In [26]:
import mlflow.sklearn

mlflow.autolog()

# with, end 구문을 붙이지 않아도 알아서 실험 종료가 됩니다.
with mlflow.start_run(nested=True): # 실험 시작
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train) # 학습 시킬 때는 학습 데이터만 제공

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"accuracy: {accuracy * 100}")

2024/08/14 15:43:11 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/08/14 15:43:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run nebulous-snail-327 at: http://127.0.0.1:5000/#/experiments/471758380148410695/runs/832b9a0b327c437caa171fb82b87f65c.
2024/08/14 15:43:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/471758380148410695.


accuracy: 96.66666666666667


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

models = {
    "LogisticRegression": LogisticRegression(
        max_iter=200, # 최대 반복 횟수
        C=1.0, # 규제 강도(C값이 작을수록 규제가 강해짐)
        solver='lbfgs', # 최적화 알고리즘
        random_state=123
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=100, # 트리의 갯수
        max_depth=None,
        random_state=123
    ),
    "SVC": SVC(
        kernel='linear', # linear, sigmoid, poly, rbf
        random_state=123 # 재연성을 위해 설정
    )
}

In [29]:
# 위 모델들을 한번에 불러와서(반복문) => 최고의 모델을 찾아내고, 해당 파라미터를 기록합니다.

mlflow.autolog()

best_accuracy = 0
best_model = None
best_model_name = None

with mlflow.start_run():
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_name = model_name
            best_model = model

        print(f"Model Name: {model_name}, Accuracy: {accuracy}")

        mlflow.log_param('best_model', best_model_name) # 파라미터 로그
        mlflow.log_metric('best_accuracy', best_accuracy) # 메트릭 로그

    print(f"Best Model Name: {best_model_name}, Best Accuracy: {best_accuracy}")

2024/08/14 15:43:19 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Model Name: LogisticRegression, Accuracy: 0.9666666666666667
Model Name: RandomForest, Accuracy: 0.9333333333333333


2024/08/14 15:44:24 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 7cfb7a2945ed494e836b9b08a219d534. Failed operations: [MlflowException("API request to http://127.0.0.1:5000/api/2.0/mlflow/runs/log-batch failed with exception HTTPConnectionPool(host=\'127.0.0.1\', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/log-batch (Caused by ResponseError(\'too many 500 error responses\'))")]')]
2024/08/14 15:44:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run abundant-ram-650 at: http://127.0.0.1:5000/#/experiments/471758380148410695/runs/7cfb7a2945ed494e836b9b08a219d534.
2024/08/14 15:44:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/471758380148410695.


Model Name: SVC, Accuracy: 0.9333333333333333
Best Model Name: LogisticRegression, Best Accuracy: 0.9666666666666667


In [30]:
mlflow.autolog()

# 전체 모델에 대해서 기록을 하고 싶은데?
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # model
        model_path = f"{model_name}_model"
        mlflow.sklearn.log_model(model, model_path) # 모델을 artifact 디렉토리에 저장

        mlflow.log_param(f'{model_name}_param', model.get_params()) # 파라미터 로그
        mlflow.log_metric(f'{model_name}_accuracy', accuracy) # 메트릭 로그

    print(f"Model Name: {model_name}, Accuracy: {accuracy}")

2024/08/14 15:44:28 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/08/14 15:44:30 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/14 15:44:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/471758380148410695/runs/a70a5e6a71b145aa8e8cf653cf89b8ae.
2024/08/14 15:44:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/471758380148410695.


Model Name: LogisticRegression, Accuracy: 0.9666666666666667


2024/08/14 15:44:33 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/14 15:44:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest at: http://127.0.0.1:5000/#/experiments/471758380148410695/runs/d11e5136b2354258a2f0bbe3f506a2ab.
2024/08/14 15:44:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/471758380148410695.


Model Name: RandomForest, Accuracy: 0.9333333333333333


2024/08/14 15:44:35 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/14 15:44:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run SVC at: http://127.0.0.1:5000/#/experiments/471758380148410695/runs/2861678abfdb42929393ba0fb2e53746.
2024/08/14 15:44:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/471758380148410695.


Model Name: SVC, Accuracy: 0.9333333333333333


# 모델 관리

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

def promote_to_staging(): pass # stage
def promote_to_production(): pass # production
def archive_model(): pass # archive: 모델 폐기 단계
def register_model(): pass # 모델 등록